In [6]:
%run MyNoteBook.py

In [1]:
import Valid_Datasets
from sklearn.decomposition import PCA
import pandas as pd

In [2]:
instance = Valid_Datasets.get_df('00EABED2-271D-49D8-B599-1D4A09240601')

test_example = instance

cleaned_data = Valid_Datasets.non_watch_value_imputer(test_example)
cleaned_data = Valid_Datasets.KNN_for_watch_data(cleaned_data,10)
cleaned_data = Valid_Datasets.pca_to_data(cleaned_data,2)
cleaned_data


ERROR:root:
UnicodeDecodeError while processing traceback.

ERROR:root:
UnicodeDecodeError while processing traceback.

ERROR:root:
UnicodeDecodeError while processing traceback.

--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\Users\xiang\AppData\Local\Programs\Python\Python38\lib\linecache.py", line 47, in getlines
    return updatecache(filename, module_globals)
  File "c:\Users\xiang\AppData\Local\Programs\Python\Python38\lib\linecache.py", line 137, in updatecache
    lines = fp.readlines()
  File "c:\Users\xiang\AppData\Local\Programs\Python\Python38\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd3 in position 4806: invalid continuation byte

Original exception was:
Traceback (most recent call last):
  File "C:\Users\xiang\AppData\Roami

: 

: 

In [ ]:
print(Valid_Datasets.get_cross_validation('test',0))

['0BFC35E2-4817-4865-BFA7-764742302A2D', '0E6184E1-90C0-48EE-B25A-F1ECB7B9714E', '1155FF54-63D3-4AB2-9863-8385D0BD0A13', '1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842', '27E04243-B138-4F40-A164-F40B60165CF3', '00EABED2-271D-49D8-B599-1D4A09240601', '098A72A5-E3E5-4F54-A152-BBDA0DF7B694', '0A986513-7828-4D53-AA1F-E02D6DF9561B', '11B5EC4D-4133-4289-B475-4E737182A406', '136562B6-95B2-483D-88DC-065F28409FD2', '1538C99F-BA1E-4EFB-A949-6C7C47701B20', '24E40C4C-A349-4F9F-93AB-01D00FB994AF']


In [ ]:
import gzip
import pandas as pd

uuid = '00EABED2-271D-49D8-B599-1D4A09240601'
user_data_file = 'Datasets/%s.features_labels.csv.gz' % uuid
with gzip.open(user_data_file,'rb') as fid:
    csv_df = pd.read_csv(fid,delimiter=',', index_col= 0)
    csv_df = pd.DataFrame(csv_df,index=pd.MultiIndex.from_product([[uuid],csv_df.index]))
    pass
csv_df

raw_acc:magnitude_stats:mean  \
                                     timestamp                                  
00EABED2-271D-49D8-B599-1D4A09240601 1444079161                           NaN   
                                     1444079221                           NaN   
                                     1444079281                           NaN   
                                     1444079341                           NaN   
                                     1444079431                           NaN   
...                                                                       ...   
                                     1444750610                           NaN   
                                     1444750670                           NaN   
                                     1444750731                           NaN   
                                     1444750768                           NaN   
                                     1444750828                           NaN   

                                                 raw_acc:magnitude_stats:std  \
                                     timestamp                                 
00EABED2-271D-49D8-B599-1D4A09240601 1444079161                          NaN   
                                     1444079221                          NaN   
                                     1444079281                          NaN   
                                     1444079341                          NaN   
                                     1444079431                          NaN   
...                                                                      ...   
                                     1444750610                          NaN   
                                     1444750670                          NaN   
                                     1444750731                          NaN   
                                     1444750768                          NaN   
                                     1444750828                          NaN   

                                                 raw_acc:magnitude_stats:moment3  \
                                     timestamp                                     
00EABED2-271D-49D8-B599-1D4A09240601 1444079161                              NaN   
                                     1444079221                              NaN   
                                     1444079281                              NaN   
                                     1444079341                              NaN   
                                     1444079431                              NaN   
...                                                                          ...   
                                     1444750610                              NaN   
                                     1444750670                              NaN   
                                     1444750731                              NaN   
                                     1444750768                              NaN   
                                     1444750828                              NaN   

                                                 raw_acc:magnitude_stats:moment4  \
                                     timestamp                                     
00EABED2-271D-49D8-B599-1D4A09240601 1444079161                              NaN   
                                     1444079221                              NaN   
                                     1444079281                              NaN   
                                     1444079341                              NaN   
                                     1444079431                              NaN   
...                                                                          ...   
                                     1444750610                              NaN   
                                     1444750670                              NaN   
                     